In [1]:
from google.colab import files
src = list(files.upload().values())[0]

import os
cwd = os.getcwd()
print(cwd)

!pip install tqdm
!pip install tensorflow-gpu==1.4

Saving helper.py to helper (1).py
Saving project_tests.py to project_tests (1).py
/content
  Using cached https://files.pythonhosted.org/packages/7d/e6/19dfaff08fcbee7f3453e5b537e65a8364f1945f921a36d08be1e2ff3475/tqdm-4.24.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 170.1MB 65kB/s 
    100% |████████████████████████████████| 1.7MB 3.7MB/s 
    100% |████████████████████████████████| 890kB 4.0MB/s 
  Running setup.py bdist_wheel for html5lib ... - done
  Stored in directory: /content/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 2.1.3
    Uninstalling bleach-2.1.3:
      Successfully uninstalled bleach-2.1.3


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [6]:
import os
cwd = os.getcwd()
print(cwd)

#os.chdir('drive/Colab_Notebooks/Semantic_Segmentation')

cwd = os.getcwd()
print(cwd)
os.listdir(os.getcwd())

/content/drive/Colab_Notebooks/Semantic_Segmentation
/content/drive/Colab_Notebooks/Semantic_Segmentation


['main2.ipynb',
 'models',
 'runs',
 'helper.py',
 'SemanticSegmentation.ipynb',
 '__pycache__',
 'data',
 'main_jupyter.ipynb',
 'main.py',
 'project_tests.py']

In [10]:
import os.path
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tqdm import tqdm
import tensorflow as tf


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    

def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights    
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    vgg_layer3_out = tf.layers.batch_normalization(vgg_layer3_out, name="new_vgg_layer3_bn")
    vgg_layer4_out = tf.layers.batch_normalization(vgg_layer4_out, name="new_vgg_layer4_bnt")
    vgg_layer7_out = tf.layers.batch_normalization(vgg_layer7_out, name="new_vgg_layer7_bn")
    
    #1x1 Convolution on layer 7
    layer7_1x1 = tf.layers.conv2d(vgg_layer7_out, filters=num_classes, kernel_size=(1, 1), strides=(1, 1),
                                          name='new_layer7_1x1_out', activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #upsampling on layer7_1x1
    layer4_upsampled =  tf.layers.conv2d_transpose(layer7_1x1, vgg_layer4_out.get_shape().as_list()[-1], 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer4_upsampled')
    
    #batch normalization on upsampled layer
    layer4_upsampled_bn = tf.layers.batch_normalization(layer4_upsampled, name="layer4_upsampled_bn")
    
    #1x1 Convolution on vgg_layer4_out   
    vgg_layer4_1x1 = tf.layers.conv2d(vgg_layer4_out, filters=vgg_layer4_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                          name="vgg_layer4_1x1", activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #batch normalization on vgg layer
    vgg_layer4_1x1_bn = tf.layers.batch_normalization(vgg_layer4_1x1, name="vgg_layer4_1x1_bn")
    
    #Final layer 4
    layer4_out = tf.add(vgg_layer4_1x1_bn, layer4_upsampled_bn, name='layer4_out')
    
    
    # Upsampled layer 3 using final layer 4
    layer3_upsampled4 = tf.layers.conv2d_transpose(layer4_out, vgg_layer3_out.get_shape().as_list()[-1], 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer3_upsampled4')
    
    #batch normalization on upsampled layer
    layer3_upsampled4_bn = tf.layers.batch_normalization(layer3_upsampled4, name="layer3_upsampled4_bn")
    
        
    #layer3 taken from a 4x upsampling from layer7_1x1
    layer3_upsampled7 = tf.layers.conv2d_transpose(layer7_1x1, vgg_layer3_out.get_shape().as_list()[-1], 8, 
                                             strides= (4, 4), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer3_upsampled7')
    
    layer3_upsampled7_bn = tf.layers.batch_normalization(layer3_upsampled7, name="layer3_upsampled7_bn")
    
    
    #1x1 Convolution on vgg_layer3_out
    vgg_layer3_1x1 = tf.layers.conv2d(vgg_layer3_out, filters=vgg_layer3_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                          name="vgg_layer3_1x1", activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #batch normalization on vgg layer
    vgg_layer3_1x1_bn = tf.layers.batch_normalization(vgg_layer3_1x1, name="vgg_layer3_1x1_bn")
    

    
    #Final Layer 3
    layer3_out = tf.add_n([vgg_layer3_1x1_bn, layer3_upsampled4_bn, layer3_upsampled7_bn], name='layer3_out')
    
    
    #Last Layer
    final_layer = tf.layers.conv2d_transpose(layer3_out, num_classes, 16,  
                                               strides= (8, 8), 
                                               padding= 'same', 
                                               kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                               kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
        
    return final_layer

tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=nn_last_layer, labels=correct_label),
                                        name="cross_entropy")
    regularization_loss = tf.losses.get_regularization_loss()
    total_loss = tf.add(cross_entropy_loss, regularization_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(total_loss)
    
    
    return logits , train_op, total_loss

tests.test_optimize(optimize)

def layers2(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    
    # Since we are not using a vanilla vgg model, there is already a 1x1 convolution on the last layer.
    # The weights for the vgg_out layers are already trained so we dont overtrain them by stopping the gradient flow.
    vgg_layer7_out = tf.stop_gradient(vgg_layer7_out)
    vgg_layer4_out = tf.stop_gradient(vgg_layer4_out)
    vgg_layer3_out = tf.stop_gradient(vgg_layer3_out)

    # Use a shorter variable name for simplicity
    layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

    # Apply 1x1 convolution in place of fully connected layer
    fcn8 = tf.layers.conv2d(layer7, filters=num_classes, kernel_size=1, name="fcn8")
    
    # Upsample fcn8 with size depth=(4096?) to match size of layer 4 so that we can add skip connection with 4th layer
    fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")
    
    # Add a skip connection between current final layer fcn8 and 4th layer
    fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")
    
    # Upsample again
    fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")
    
    # Add skip connection
    fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")
    
    # Upsample again
    fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=num_classes,
    kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

    return fcn11



def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())

    
    print("Training...")
    for i in tqdm(range(epochs)):
        print("EPOCH {} ...".format(i+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0005})
        print("Loss: = {:.3f}".format(loss))
    
    print("Training Finished!!")
    


def run():
    
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs2'
    model_dir = './models2'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    
    ## gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
            
    tf.reset_default_graph()

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        ## vgg_path = '.\\data\\vgg'
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)


        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        
        epochs = 20
        batch_size = 5

        # TF placeholders
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')

        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)

        nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        ## sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate)
        
        ## sess.run(tf.global_variables_initializer())
        ## saver = tf.train.Saver()

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(model_dir, runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image, saver)
        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()
print("Done")


TensorFlow Version: 1.10.0-rc1
Default GPU Device: /device:GPU:0
Tests Passed
Tests Passed
Tests Passed
Tests Passed
INFO:tensorflow:Restoring parameters from ./data/vgg/variables/variables


  0%|          | 0/20 [00:00<?, ?it/s]

Training...
EPOCH 1 ...


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
  5%|▌         | 1/20 [08:56<2:50:02, 536.96s/it]

Loss: = 0.503
EPOCH 2 ...


...

 95%|█████████▌| 19/20 [30:48<01:37, 97.30s/it]

Loss: = 0.163
EPOCH 20 ...


100%|██████████| 20/20 [32:01<00:00, 96.08s/it]

Loss: = 0.133
Training Finished!!


Training Finished. Saving test images to: ./runs2/1534107601.6213937
Done
